In [3]:
# lstm_lstm: emotion recognition from speech= lstm, text=lstm
# created for ATSIT paper 2020
# coded by Bagus Tris Atmaja (bagus@ep.its.ac.id)

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from gensim.models.keyedvectors import KeyedVectors
from keras.layers.convolutional import Conv2D

/root/anaconda3/envs/ser36/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [5]:

# text feature and labels
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
# x_train_text = np.load(path+'x_train_text.npy')
# vad = np.load(path+'y_egemaps.npy')
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')

data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)
# standardization
scaled_feature = False

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
EMBEDDING_DIM = 300
word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

file_loc = '../../dimensionalSM/data/GoogleNews-vectors-negative300.bin'
print (file_loc)

word_vectors = KeyedVectors.load_word2vec_format(file_loc, binary=True)

g_word_embedding_matrix = {}
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=nb_words:
        continue
    try:
        gembedding_vector = word_vectors[word]
        g_word_embedding_matrix[i] = gembedding_vector
    except KeyError:
        g_word_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))
del(word_vectors)



scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# split train/test

earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)



Found 3437 unique tokens
../../dimensionalSM/data/GoogleNews-vectors-negative300.bin
G Null word embeddings: 1


In [6]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [7]:
split = 7869

In [8]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [9]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(input_mocap)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    # combined model
    model_combined = concatenate([model_speech, model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


#model_1.summary()

In [10]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split],x_train_mocap[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:],x_train_mocap[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('stmW2V_multi7869-3.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 71s 11ms/step - loss: 1.8618 - v_loss: 0.7503 - a_loss: 0.5170 - d_loss: 0.5933 - v_ccc: 0.2491 - a_ccc: 0.4826 - d_ccc: 0.4065 - val_loss: 2.3117 - val_v_loss: 0.7664 - val_a_loss: 0.7871 - val_d_loss: 0.7653 - val_v_ccc: 0.2398 - val_a_ccc: 0.2146 - val_d_ccc: 0.2339
Epoch 2/50
6295/6295 [==============================] - 66s 11ms/step - loss: 1.1557 - v_loss: 0.4011 - a_loss: 0.3306 - d_loss: 0.4233 - v_ccc: 0.5987 - a_ccc: 0.6693 - d_ccc: 0.5764 - val_loss: 1.6212 - val_v_loss: 0.6093 - val_a_loss: 0.4827 - val_d_loss: 0.5391 - val_v_ccc: 0.3958 - val_a_ccc: 0.5219 - val_d_ccc: 0.4611
Epoch 3/50
6295/6295 [==============================] - 66s 10ms/step - loss: 1.0190 - v_loss: 0.3265 - a_loss: 0.3059 - d_loss: 0.3870 - v_ccc: 0.6740 - a_ccc: 0.6940 - d_ccc: 0.6131 - val_loss: 1.5325 - val_v_loss: 0

Epoch 10/50
6295/6295 [==============================] - 65s 10ms/step - loss: 0.5539 - v_loss: 0.1397 - a_loss: 0.1948 - d_loss: 0.2192 - v_ccc: 0.8602 - a_ccc: 0.8052 - d_ccc: 0.7807 - val_loss: 1.5578 - val_v_loss: 0.5261 - val_a_loss: 0.4575 - val_d_loss: 0.5850 - val_v_ccc: 0.4761 - val_a_ccc: 0.5454 - val_d_ccc: 0.4206
Epoch 11/50
6295/6295 [==============================] - 66s 10ms/step - loss: 0.5241 - v_loss: 0.1337 - a_loss: 0.1845 - d_loss: 0.2060 - v_ccc: 0.8662 - a_ccc: 0.8155 - d_ccc: 0.7942 - val_loss: 1.5578 - val_v_loss: 0.5396 - val_a_loss: 0.4706 - val_d_loss: 0.5760 - val_v_ccc: 0.4710 - val_a_ccc: 0.5392 - val_d_ccc: 0.4320
Epoch 12/50
6295/6295 [==============================] - 65s 10ms/step - loss: 0.4841 - v_loss: 0.1246 - a_loss: 0.1731 - d_loss: 0.1867 - v_ccc: 0.8756 - a_ccc: 0.8270 - d_ccc: 0.8134 - val_loss: 1.5745 - val_v_loss: 0.5351 - val_a_loss: 0.4804 - val_d_loss: 0.5733 - val_v_ccc: 0.4747 - val_a_ccc: 0.5202 - val_d_ccc: 0.4306
Epoch 13/50
6295/62

6295/6295 [==============================] - 65s 10ms/step - loss: 0.9233 - v_loss: 0.2632 - a_loss: 0.2957 - d_loss: 0.3642 - v_ccc: 0.7367 - a_ccc: 0.7042 - d_ccc: 0.6359 - val_loss: 1.5308 - val_v_loss: 0.5544 - val_a_loss: 0.4335 - val_d_loss: 0.5555 - val_v_ccc: 0.4495 - val_a_ccc: 0.5697 - val_d_ccc: 0.4501
Epoch 5/50
6295/6295 [==============================] - 64s 10ms/step - loss: 0.8386 - v_loss: 0.2228 - a_loss: 0.2771 - d_loss: 0.3386 - v_ccc: 0.7774 - a_ccc: 0.7228 - d_ccc: 0.6612 - val_loss: 1.5322 - val_v_loss: 0.5458 - val_a_loss: 0.4230 - val_d_loss: 0.5800 - val_v_ccc: 0.4615 - val_a_ccc: 0.5818 - val_d_ccc: 0.4246
Epoch 6/50
6295/6295 [==============================] - 64s 10ms/step - loss: 0.7699 - v_loss: 0.2025 - a_loss: 0.2636 - d_loss: 0.3047 - v_ccc: 0.7979 - a_ccc: 0.7369 - d_ccc: 0.6953 - val_loss: 1.7439 - val_v_loss: 0.7039 - val_a_loss: 0.5059 - val_d_loss: 0.5481 - val_v_ccc: 0.3027 - val_a_ccc: 0.4967 - val_d_ccc: 0.4566
Epoch 7/50
6295/6295 [===========

Epoch 10/50
6295/6295 [==============================] - 62s 10ms/step - loss: 0.5802 - v_loss: 0.1493 - a_loss: 0.2027 - d_loss: 0.2284 - v_ccc: 0.8507 - a_ccc: 0.7973 - d_ccc: 0.7717 - val_loss: 1.4868 - val_v_loss: 0.5305 - val_a_loss: 0.4357 - val_d_loss: 0.5347 - val_v_ccc: 0.4791 - val_a_ccc: 0.5643 - val_d_ccc: 0.4697
Epoch 11/50
6295/6295 [==============================] - 62s 10ms/step - loss: 0.5449 - v_loss: 0.1376 - a_loss: 0.1937 - d_loss: 0.2136 - v_ccc: 0.8623 - a_ccc: 0.8064 - d_ccc: 0.7864 - val_loss: 1.5057 - val_v_loss: 0.5104 - val_a_loss: 0.4580 - val_d_loss: 0.5505 - val_v_ccc: 0.4965 - val_a_ccc: 0.5431 - val_d_ccc: 0.4548
Epoch 12/50
6295/6295 [==============================] - 62s 10ms/step - loss: 0.5077 - v_loss: 0.1324 - a_loss: 0.1812 - d_loss: 0.1948 - v_ccc: 0.8676 - a_ccc: 0.8193 - d_ccc: 0.8054 - val_loss: 1.5581 - val_v_loss: 0.5446 - val_a_loss: 0.4665 - val_d_loss: 0.5591 - val_v_ccc: 0.4621 - val_a_ccc: 0.5356 - val_d_ccc: 0.4443
Epoch 13/50
6295/62

6295/6295 [==============================] - 65s 10ms/step - loss: 2.0334 - v_loss: 0.7472 - a_loss: 0.6054 - d_loss: 0.6802 - v_ccc: 0.2525 - a_ccc: 0.3944 - d_ccc: 0.3198 - val_loss: 2.0289 - val_v_loss: 0.7275 - val_a_loss: 0.6962 - val_d_loss: 0.6173 - val_v_ccc: 0.2806 - val_a_ccc: 0.3063 - val_d_ccc: 0.3842
Epoch 2/50
6295/6295 [==============================] - 60s 9ms/step - loss: 1.1354 - v_loss: 0.3779 - a_loss: 0.3328 - d_loss: 0.4247 - v_ccc: 0.6220 - a_ccc: 0.6672 - d_ccc: 0.5754 - val_loss: 1.6535 - val_v_loss: 0.6537 - val_a_loss: 0.4766 - val_d_loss: 0.5355 - val_v_ccc: 0.3530 - val_a_ccc: 0.5276 - val_d_ccc: 0.4659
Epoch 3/50
6295/6295 [==============================] - 59s 9ms/step - loss: 0.9907 - v_loss: 0.2941 - a_loss: 0.3104 - d_loss: 0.3861 - v_ccc: 0.7058 - a_ccc: 0.6898 - d_ccc: 0.6137 - val_loss: 1.5943 - val_v_loss: 0.6417 - val_a_loss: 0.4196 - val_d_loss: 0.5437 - val_v_ccc: 0.3657 - val_a_ccc: 0.5832 - val_d_ccc: 0.4567
Epoch 4/50
6295/6295 [=============

...

2170/2170 [==============================] - 5s 2ms/step
[0.44504010677337646, 0.5797367095947266, 0.45772647857666016]
23
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 76s 12ms/step - loss: 2.0600 - v_loss: 0.7597 - a_loss: 0.6161 - d_loss: 0.6824 - v_ccc: 0.2397 - a_ccc: 0.3833 - d_ccc: 0.3170 - val_loss: 2.0542 - val_v_loss: 0.7653 - val_a_loss: 0.6729 - val_d_loss: 0.6321 - val_v_ccc: 0.2401 - val_a_ccc: 0.3321 - val_d_ccc: 0.3736
Epoch 2/50
6295/6295 [==============================] - 59s 9ms/step - loss: 1.1673 - v_loss: 0.3953 - a_loss: 0.3346 - d_loss: 0.4369 - v_ccc: 0.6046 - a_ccc: 0.6652 - d_ccc: 0.5630 - val_loss: 1.7529 - val_v_loss: 0.6240 - val_a_loss: 0.5232 - val_d_loss: 0.6182 - val_v_ccc: 0.3815 - val_a_ccc: 0.4816 - val_d_ccc: 0.3839
Epoch 3/50
6295/6295 [==============================] - 59s 9ms/step - loss: 1.0350 - v_loss: 0.3236 - a_loss: 0.3166 - d_loss: 0.3949 - v_ccc: 0.6766 - a_ccc: 0.6833 - d_ccc: 0.

6295/6295 [==============================] - 65s 10ms/step - loss: 1.0181 - v_loss: 0.3278 - a_loss: 0.3036 - d_loss: 0.3872 - v_ccc: 0.6726 - a_ccc: 0.6964 - d_ccc: 0.6129 - val_loss: 1.5479 - val_v_loss: 0.5800 - val_a_loss: 0.4191 - val_d_loss: 0.5620 - val_v_ccc: 0.4244 - val_a_ccc: 0.5883 - val_d_ccc: 0.4394
Epoch 4/50
6295/6295 [==============================] - 65s 10ms/step - loss: 0.9321 - v_loss: 0.2794 - a_loss: 0.2932 - d_loss: 0.3593 - v_ccc: 0.7206 - a_ccc: 0.7067 - d_ccc: 0.6406 - val_loss: 1.5133 - val_v_loss: 0.5659 - val_a_loss: 0.4368 - val_d_loss: 0.5286 - val_v_ccc: 0.4428 - val_a_ccc: 0.5675 - val_d_ccc: 0.4764
Epoch 5/50
6295/6295 [==============================] - 65s 10ms/step - loss: 0.8479 - v_loss: 0.2352 - a_loss: 0.2748 - d_loss: 0.3378 - v_ccc: 0.7647 - a_ccc: 0.7252 - d_ccc: 0.6622 - val_loss: 1.5079 - val_v_loss: 0.5606 - val_a_loss: 0.4136 - val_d_loss: 0.5407 - val_v_ccc: 0.4408 - val_a_ccc: 0.5925 - val_d_ccc: 0.4587
Epoch 6/50
6295/6295 [===========

Epoch 12/50
6295/6295 [==============================] - 66s 10ms/step - loss: 0.4918 - v_loss: 0.1268 - a_loss: 0.1711 - d_loss: 0.1940 - v_ccc: 0.8734 - a_ccc: 0.8289 - d_ccc: 0.8059 - val_loss: 1.4678 - val_v_loss: 0.4918 - val_a_loss: 0.4572 - val_d_loss: 0.5320 - val_v_ccc: 0.5150 - val_a_ccc: 0.5451 - val_d_ccc: 0.4721
Epoch 13/50
6295/6295 [==============================] - 66s 10ms/step - loss: 0.4651 - v_loss: 0.1186 - a_loss: 0.1637 - d_loss: 0.1831 - v_ccc: 0.8814 - a_ccc: 0.8366 - d_ccc: 0.8170 - val_loss: 1.5687 - val_v_loss: 0.5215 - val_a_loss: 0.4826 - val_d_loss: 0.5826 - val_v_ccc: 0.4842 - val_a_ccc: 0.5219 - val_d_ccc: 0.4252
Epoch 14/50
2170/2170 [==============================] - 6s 3ms/step
[0.4646777808666229, 0.572455883026123, 0.4701419472694397]
26
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 85s 13ms/step - loss: 1.9912 - v_loss: 0.7597 - a_loss: 0.5828 - d_loss: 0.6474 - v_ccc: 0.2398 - a_ccc: 0.416

6295/6295 [==============================] - 65s 10ms/step - loss: 0.8321 - v_loss: 0.2258 - a_loss: 0.2750 - d_loss: 0.3314 - v_ccc: 0.7744 - a_ccc: 0.7251 - d_ccc: 0.6684 - val_loss: 1.5453 - val_v_loss: 0.5307 - val_a_loss: 0.4833 - val_d_loss: 0.5402 - val_v_ccc: 0.4751 - val_a_ccc: 0.5151 - val_d_ccc: 0.4645
Epoch 6/50
6295/6295 [==============================] - 66s 10ms/step - loss: 0.7760 - v_loss: 0.1995 - a_loss: 0.2635 - d_loss: 0.3133 - v_ccc: 0.8009 - a_ccc: 0.7364 - d_ccc: 0.6867 - val_loss: 1.5780 - val_v_loss: 0.5993 - val_a_loss: 0.4341 - val_d_loss: 0.5643 - val_v_ccc: 0.4096 - val_a_ccc: 0.5679 - val_d_ccc: 0.4445
Epoch 7/50
6295/6295 [==============================] - 66s 11ms/step - loss: 0.7199 - v_loss: 0.1841 - a_loss: 0.2465 - d_loss: 0.2893 - v_ccc: 0.8161 - a_ccc: 0.7534 - d_ccc: 0.7106 - val_loss: 1.5094 - val_v_loss: 0.5215 - val_a_loss: 0.4520 - val_d_loss: 0.5550 - val_v_ccc: 0.4851 - val_a_ccc: 0.5539 - val_d_ccc: 0.4517
Epoch 8/50
6295/6295 [===========

Epoch 11/50
6295/6295 [==============================] - 64s 10ms/step - loss: 0.5307 - v_loss: 0.1341 - a_loss: 0.1846 - d_loss: 0.2118 - v_ccc: 0.8659 - a_ccc: 0.8153 - d_ccc: 0.7881 - val_loss: 1.6007 - val_v_loss: 0.5518 - val_a_loss: 0.4830 - val_d_loss: 0.5849 - val_v_ccc: 0.4567 - val_a_ccc: 0.5211 - val_d_ccc: 0.4215
Epoch 12/50
6295/6295 [==============================] - 64s 10ms/step - loss: 0.5021 - v_loss: 0.1283 - a_loss: 0.1781 - d_loss: 0.1960 - v_ccc: 0.8717 - a_ccc: 0.8220 - d_ccc: 0.8042 - val_loss: 1.5583 - val_v_loss: 0.5387 - val_a_loss: 0.4679 - val_d_loss: 0.5725 - val_v_ccc: 0.4714 - val_a_ccc: 0.5324 - val_d_ccc: 0.4379
Epoch 13/50
6295/6295 [==============================] - 64s 10ms/step - loss: 0.4655 - v_loss: 0.1192 - a_loss: 0.1642 - d_loss: 0.1820 - v_ccc: 0.8808 - a_ccc: 0.8358 - d_ccc: 0.8179 - val_loss: 1.5874 - val_v_loss: 0.5375 - val_a_loss: 0.4923 - val_d_loss: 0.5903 - val_v_ccc: 0.4730 - val_a_ccc: 0.5124 - val_d_ccc: 0.4271
Epoch 14/50
6295/62